**Preprocessing and Low-Pass Filtering of EDA Signals**

In [121]:
# ============================================================
# EDA FILTERING BY PARTS (Embrace EDA)
# - Preserves part01.csv, part02.csv, ...
# - Stores filtered output in EDA_filtrado
# ============================================================

import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt


# ------------------------------------------------------------
# Configuration
# ------------------------------------------------------------
base_dir = os.path.expanduser("~/TFM/2.step2_e4csv")
fs = 4          # Sampling frequency (Hz)
cutoff = 0.05   # Low-pass cutoff frequency (Hz)


# ------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------
def butter_lowpass_filter(data, cutoff, fs, order=4):
    """
    Applies a Butterworth low-pass filter to the signal.
    """
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype="low", analog=False)
    return filtfilt(b, a, data)


def remove_spikes(series, z_thresh=5):
    """
    Removes extreme spikes using a local z-score threshold
    and interpolates missing values.
    """
    s = series.copy()
    mean = s.rolling(200, min_periods=1, center=True).mean()
    std = s.rolling(200, min_periods=1, center=True).std()
    z = (s - mean) / (std + 1e-9)
    s[np.abs(z) > z_thresh] = np.nan
    return s.interpolate(limit_direction="both")


def procesar_eda_por_partes(embrace_id):
    """
    Filters all EDA parts for a given Embrace subject
    and saves them using the original E4 CSV format.
    """
    eda_dir = os.path.join(base_dir, embrace_id, "EDA")
    out_dir = os.path.join(base_dir, embrace_id, "EDA_filtrado")
    os.makedirs(out_dir, exist_ok=True)

    print(f"\nProcessing subject {embrace_id}...")
    all_files = sorted([f for f in os.listdir(eda_dir) if f.endswith(".csv")])
    total_samples = 0

    for fname in all_files:
        path = os.path.join(eda_dir, fname)

        # Read E4-format CSV (timestamp, fs, values)
        df = pd.read_csv(path, header=None)
        start_time = float(df.iloc[0, 0])
        freq = float(df.iloc[1, 0])
        vals = df.iloc[2:, 0].astype(float).reset_index(drop=True)

        # Spike removal and low-pass filtering
        clean = remove_spikes(vals)
        filtered = butter_lowpass_filter(clean, cutoff=cutoff, fs=fs)

        # Write filtered signal preserving E4 structure
        out_path = os.path.join(out_dir, fname)
        with open(out_path, "w") as f:
            f.write(f"{start_time:.6f}\n")
            f.write(f"{freq:.6f}\n")
            for val in filtered:
                f.write(f"{val:.9f}\n")

        total_samples += len(filtered)
        print(f"Saved {fname} ({len(filtered)} samples)")

    # Save basic processing metrics
    with open(os.path.join(out_dir, "metrics.txt"), "w") as f:
        f.write(f"Filtered EDA for subject {embrace_id}\n")
        f.write(f"Sampling frequency: {fs} Hz\n")
        f.write(f"Low-pass cutoff frequency: {cutoff} Hz\n")
        f.write(f"Processed parts: {len(all_files)}\n")
        f.write(f"Total samples: {total_samples}\n")

    print(f"Metrics saved in {out_dir}/metrics.txt")


# ------------------------------------------------------------
# Process all subjects
# ------------------------------------------------------------
def procesar_todos():
    sujetos = [
        "CLIDEM01", "CLIDEM010", "CLIDEM011", "CLIDEM02", "CLIDEM030",
        "CLIDEM031", "CLIDEM04", "CLIDEM05", "CLIDEM06", "CLIDEM07",
        "CLIDEM08", "CLIDEM09", "CLIDEM11", "CLIDEM12", "CLIDEM13",
        "CLIDEM14", "CLIDEM15", "CLIDEM16", "CLIDEM17", "CLIDEM18",
        "CLIDEM19", "CLIDEM20", "CLIDEM21", "CLIDEM22", "CLIDEM23",
        "CLIDEM24", "CLIDEM25", "CLIDEM26", "CLIDEM27", "CLIDEM29"
    ]

    for s in sujetos:
        try:
            procesar_eda_por_partes(s)
        except Exception as e:
            print(f"Error processing {s}: {e}")


# ------------------------------------------------------------
# Main entry point
# ------------------------------------------------------------
if __name__ == "__main__":
    procesar_todos()


🔹 Procesando CLIDEM01...
✅ Guardado part01.csv (527250 muestras)
✅ Guardado part02.csv (66690 muestras)
🧾 Métricas guardadas en /home/yasmina/TFM/2.step2_e4csv/CLIDEM01/EDA_filtrado/metrics.txt

🔹 Procesando CLIDEM010...
✅ Guardado part01.csv (109900 muestras)
✅ Guardado part02.csv (626460 muestras)
🧾 Métricas guardadas en /home/yasmina/TFM/2.step2_e4csv/CLIDEM010/EDA_filtrado/metrics.txt

🔹 Procesando CLIDEM011...
✅ Guardado part01.csv (663870 muestras)
🧾 Métricas guardadas en /home/yasmina/TFM/2.step2_e4csv/CLIDEM011/EDA_filtrado/metrics.txt

🔹 Procesando CLIDEM02...
✅ Guardado part01.csv (828380 muestras)
✅ Guardado part02.csv (30 muestras)
✅ Guardado part03.csv (170 muestras)
✅ Guardado part04.csv (39710 muestras)
🧾 Métricas guardadas en /home/yasmina/TFM/2.step2_e4csv/CLIDEM02/EDA_filtrado/metrics.txt

🔹 Procesando CLIDEM030...
✅ Guardado part01.csv (686082 muestras)
✅ Guardado part02.csv (143150 muestras)
🧾 Métricas guardadas en /home/yasmina/TFM/2.step2_e4csv/CLIDEM030/EDA_filt

**ACC Processing Pipeline**

In [127]:
# ============================================================
# MASS ACC PROCESSING (Embrace Plus vs Empatica)
# - Automatic lag estimation
# - Clipping to ±128
# - Per-part processing and saving
# ============================================================

import os
import numpy as np
import pandas as pd
from glob import glob
from scipy.stats import pearsonr
from tqdm import tqdm


# ============================================================
# Helper functions
# ============================================================

def load_acc_parts(folder):
    """
    Load all ACC parts (partXX.csv) from an Embrace folder.
    Returns a list of (part_name, sampling_freq, dataframe).
    """
    parts = []
    for path in sorted(glob(os.path.join(folder, "part*.csv"))):
        df = pd.read_csv(path, header=None)

        start_time = float(df.iloc[0, 0])
        freq = float(df.iloc[1, 0])
        vals = df.iloc[2:, :3].astype(float).values

        ts = start_time + np.arange(len(vals)) / freq

        df_part = pd.DataFrame({
            "timestamp": pd.to_datetime(ts, unit="s", utc=True),
            "x": vals[:, 0],
            "y": vals[:, 1],
            "z": vals[:, 2]
        })

        parts.append((os.path.basename(path), freq, df_part))

    return parts


def load_empatica_acc(path):
    """
    Load Empatica ACC file (single CSV).
    """
    df = pd.read_csv(path, header=None)

    start_time = float(df.iloc[0, 0])
    freq = float(df.iloc[1, 0])
    vals = df.iloc[2:, :3].astype(float).values

    ts = start_time + np.arange(len(vals)) / freq

    df = pd.DataFrame({
        "timestamp": pd.to_datetime(ts, unit="s", utc=True),
        "x": vals[:, 0],
        "y": vals[:, 1],
        "z": vals[:, 2]
    })

    df.set_index("timestamp", inplace=True)
    return df, freq


def clip_acc(df):
    """
    Clip acceleration values to the range [-128, 128].
    """
    for axis in ["x", "y", "z"]:
        df[axis] = np.clip(df[axis], -128, 128)
    return df


def estimate_lag(x, y, max_lag_s=60):
    """
    Estimate lag (in seconds) maximizing correlation between two signals.
    """
    n = min(len(x), len(y))
    x, y = x[:n], y[:n]

    lags = np.arange(-max_lag_s, max_lag_s + 1, 1)
    corrs = []

    for lag in lags:
        y_shift = np.roll(y, lag)
        valid = ~np.isnan(x) & ~np.isnan(y_shift)

        if valid.sum() > 100:
            corrs.append(np.corrcoef(x[valid], y_shift[valid])[0, 1])
        else:
            corrs.append(np.nan)

    best_idx = np.nanargmax(corrs)
    return lags[best_idx], corrs[best_idx]


def save_filtered_part(part_name, freq, df, out_dir):
    """
    Save ACC part using Empatica-compatible CSV format.
    """
    os.makedirs(out_dir, exist_ok=True)

    start_time = df["timestamp"].iloc[0].timestamp()
    out_path = os.path.join(out_dir, part_name)

    with open(out_path, "w") as f:
        # First line: start timestamp repeated three times
        f.write(f"{start_time:.6f},{start_time:.6f},{start_time:.6f}\n")
        # Second line: sampling frequency repeated three times
        f.write(f"{freq:.6f},{freq:.6f},{freq:.6f}\n")
        # Data rows: x, y, z
        df[["x", "y", "z"]].to_csv(f, header=False, index=False)

    print(f"Saved {out_path} ({len(df)} samples)")


# ============================================================
# Single-subject ACC processing
# ============================================================

def process_acc_subject(
    embrace_id,
    empatica_id,
    base_embrace="~/TFM/2.step2_e4csv",
    base_empatica="~/TFM/EMPATICA_all"
):
    base_embrace = os.path.expanduser(base_embrace)
    base_empatica = os.path.expanduser(base_empatica)

    embrace_path = os.path.join(base_embrace, embrace_id, "ACC")
    empatica_path = os.path.join(base_empatica, empatica_id, "ACC.csv")

    # Load Embrace ACC parts
    parts = load_acc_parts(embrace_path)

    # Load Empatica ACC
    acc_empatica, _ = load_empatica_acc(empatica_path)

    # Concatenate all Embrace parts for global lag estimation
    all_embrace = pd.concat([p[2] for p in parts], ignore_index=True)
    all_embrace.set_index("timestamp", inplace=True)

    # Compute magnitude signals
    acc_empatica["mag"] = np.sqrt(
        acc_empatica["x"]**2 + acc_empatica["y"]**2 + acc_empatica["z"]**2
    )
    all_embrace["mag"] = np.sqrt(
        all_embrace["x"]**2 + all_embrace["y"]**2 + all_embrace["z"]**2
    )

    # Resample both magnitudes to 1 Hz
    acc_empatica_r = acc_empatica["mag"].resample("1s").mean().interpolate()
    acc_embrace_r = all_embrace["mag"].resample("1s").mean().interpolate()

    # Estimate global lag
    best_lag, best_corr = estimate_lag(
        acc_empatica_r.to_numpy(),
        acc_embrace_r.to_numpy(),
        max_lag_s=60
    )

    print(f"{embrace_id}: Lag={best_lag:+.2f}s, Corr={best_corr:.3f}")

    # Apply lag and save each ACC part
    out_dir = os.path.join(base_embrace, embrace_id, "ACC_filtrado")
    for part_name, freq, df in parts:
        df = clip_acc(df.copy())
        df["timestamp"] = df["timestamp"] + pd.to_timedelta(best_lag, unit="s")
        save_filtered_part(part_name, freq, df, out_dir)

    # Save metrics
    metrics_path = os.path.join(out_dir, "metrics.txt")
    with open(metrics_path, "w") as f:
        f.write(f"Lag applied: {best_lag:.3f} s\n")
        f.write(f"Global correlation: {best_corr:.3f}\n")

    return {
        "Embrace": embrace_id,
        "Empatica": empatica_id,
        "Lag (s)": best_lag,
        "Correlation": best_corr
    }


# ============================================================
# Batch processing for all subjects
# ============================================================

def process_all():
    emp_emp_map = {
        "CLIDEM01": "1823370", "CLIDEM010": "1842113", "CLIDEM011": "1891004",
        "CLIDEM02": "1832743", "CLIDEM030": "1916330", "CLIDEM031": "1920720",
        "CLIDEM04": "1833368", "CLIDEM05": "1837341", "CLIDEM06": "1840660",
        "CLIDEM07": "1840605", "CLIDEM08": "1841430", "CLIDEM09": "1842162",
        "CLIDEM11": "1891004", "CLIDEM12": "1891236", "CLIDEM13": "1895868",
        "CLIDEM14": "1891785", "CLIDEM15": "1894131", "CLIDEM16": "1895849",
        "CLIDEM17": "1896649", "CLIDEM18": "1898066", "CLIDEM19": "1900189",
        "CLIDEM20": "1906938", "CLIDEM21": "1907236", "CLIDEM22": "1908525",
        "CLIDEM23": "1909384", "CLIDEM24": "1910353", "CLIDEM25": "1911867",
        "CLIDEM26": "1912600", "CLIDEM27": "1913365", "CLIDEM29": "1914654"
    }

    results = []
    print("Processing ACC for all subjects")

    for embrace_id, empatica_id in tqdm(emp_emp_map.items(), desc="Overall progress", ncols=90):
        try:
            res = process_acc_subject(embrace_id, empatica_id)
            results.append(res)
        except Exception as e:
            print(f"Error processing {embrace_id}: {e}")

    summary_path = os.path.expanduser("~/TFM/2.step2_e4csv/ACC_summary_all.txt")
    pd.DataFrame(results).to_csv(summary_path, sep="\t", index=False)

    print(f"Global summary saved to {summary_path}")


# ============================================================
# Main entry point
# ============================================================

if __name__ == "__main__":
    process_all()


🚀 Procesando ACC para todos los sujetos...



Progreso general:   0%|                                            | 0/30 [00:00<?, ?it/s]


⏱️ CLIDEM01: Lag=-59.00s, Corr=0.140


Progreso general:   3%|█▏                                  | 1/30 [00:13<06:30, 13.45s/it]

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM01/ACC_filtrado/part01.csv (4752719 muestras)

⏱️ CLIDEM010: Lag=-8.00s, Corr=-0.004
✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM010/ACC_filtrado/part01.csv (879152 muestras)


...

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM26/ACC_filtrado/part02.csv (2920064 muestras)

⏱️ CLIDEM27: Lag=+60.00s, Corr=0.501


Progreso general:  97%|█████████████████████████████████▊ | 29/30 [07:53<00:16, 16.61s/it]

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM27/ACC_filtrado/part01.csv (4981008 muestras)

⏱️ CLIDEM29: Lag=-4.00s, Corr=0.298


Progreso general: 100%|███████████████████████████████████| 30/30 [08:13<00:00, 16.44s/it]

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM29/ACC_filtrado/part01.csv (6869319 muestras)

📊 Resumen global guardado en /home/yasmina/TFM/2.step2_e4csv/ACC_summary_all.txt


**IBI Processing and Alignment**

In [128]:
# ============================================================
# MASS IBI PROCESSING (Empatica vs Embrace)
# - Automatic lag detection
# - Pairing and filtering of IBI events
# - Metrics computation
# - CSV, TXT and global summary export
# ============================================================

import os
import numpy as np
import pandas as pd
from glob import glob
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import warnings

# Silence pandas future warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# ============================================================
# AUXILIARY FUNCTIONS
# ============================================================

def load_ibi_empatica(path):
    """Load Empatica IBI file and return absolute timestamps."""
    data = pd.read_csv(path, header=None, names=["col1", "col2"])
    start_time = float(data.iloc[0, 0])
    ibi_data = data.iloc[1:].astype(float).reset_index(drop=True)
    ibi_data.columns = ["timestamp_rel", "IBI"]
    ibi_data["timestamp_abs"] = start_time + ibi_data["timestamp_rel"]
    ibi_data["time"] = pd.to_datetime(ibi_data["timestamp_abs"], unit="s", utc=True)
    return ibi_data[["time", "IBI"]]


def load_ibi_embrace(ibi_dir):
    """Load and concatenate all Embrace IBI parts."""
    ibi_files = sorted(glob(os.path.join(ibi_dir, "part*.csv")))
    if not ibi_files:
        raise FileNotFoundError(f"No partXX.csv files found in {ibi_dir}")

    dfs = []
    for path in ibi_files:
        raw = pd.read_csv(path, header=None)
        start_time = float(raw.iloc[0, 0])
        ibi_data = raw.iloc[1:].astype(float)
        ibi_data.columns = ["timestamp_rel", "IBI"]
        ibi_data["timestamp_abs"] = start_time + ibi_data["timestamp_rel"]
        ibi_data["time"] = pd.to_datetime(ibi_data["timestamp_abs"], unit="s", utc=True)
        dfs.append(ibi_data[["time", "IBI"]])

    return pd.concat(dfs, ignore_index=True).dropna()


def hampel(series, k=4, nsigma=3.0):
    """Hampel filter for outlier correction."""
    x = series.to_numpy().copy()
    y = x.copy()
    n = len(x)

    for i in range(n):
        a, b = max(0, i - k), min(n, i + k + 1)
        med = np.median(x[a:b])
        mad = np.median(np.abs(x[a:b] - med)) + 1e-12
        if abs(x[i] - med) > nsigma * 1.4826 * mad:
            y[i] = med

    return pd.Series(y, index=series.index)


def moving_median(s, win=5):
    """Apply moving median smoothing."""
    return s.rolling(win, center=True, min_periods=1).median()


# ============================================================
# GLOBAL LAG ESTIMATION
# ============================================================

def estimate_lag(df_emp, df_embr, max_lag_s=60, step_s=0.5):
    """Estimate lag maximizing Pearson correlation."""
    tmin = max(df_emp["time"].min(), df_embr["time"].min())
    tmax = min(df_emp["time"].max(), df_embr["time"].max())
    if tmax <= tmin:
        raise ValueError("Insufficient temporal overlap")

    emp_r = df_emp.set_index("time").resample("1s").mean().interpolate()["IBI"]
    embr_r = df_embr.set_index("time").resample("1s").mean().interpolate()["IBI"]

    emp_r = emp_r.loc[tmin:tmax]
    embr_r = embr_r.loc[tmin:tmax]

    lags = np.arange(-max_lag_s, max_lag_s + step_s, step_s)
    corrs = []

    for lag in lags:
        shifted = embr_r.shift(int(lag))
        valid = shifted.notna() & emp_r.notna()
        if valid.sum() > 10:
            corrs.append(pearsonr(emp_r[valid], shifted[valid])[0])
        else:
            corrs.append(np.nan)

    best_idx = np.nanargmax(corrs)
    return lags[best_idx], corrs[best_idx]


# ============================================================
# EVENT MATCHING AND FILTERING
# ============================================================

def match_and_filter(df_emp, df_embr, frac_tol=0.20, ratio_tol=0.15, lag_sec=0.0):
    """Match IBI events using temporal and amplitude tolerances."""
    embr = df_embr.copy()
    embr["time"] = embr["time"] + pd.to_timedelta(lag_sec, unit="s")

    emp_t = df_emp["time"].astype("int64") / 1e9
    emp_ibi = df_emp["IBI"].to_numpy()
    embr_t = embr["time"].astype("int64") / 1e9
    embr_ibi = embr["IBI"].to_numpy()

    i, j = 0, 0
    idx_emp, idx_embr = [], []

    while i < len(emp_t) and j < len(embr_t):
        dt = embr_t[j] - emp_t[i]
        tol_t = max(0.05, frac_tol * emp_ibi[i])

        if abs(dt) <= tol_t:
            if abs(embr_ibi[j] - emp_ibi[i]) <= ratio_tol * emp_ibi[i]:
                idx_emp.append(i)
                idx_embr.append(j)
            i += 1
            j += 1
        elif dt < -tol_t:
            j += 1
        else:
            i += 1

    return df_emp.iloc[idx_emp].reset_index(drop=True), embr.iloc[idx_embr].reset_index(drop=True)


# ============================================================
# SAVE RESULTS
# ============================================================

def guardar_resultados(df_embr_m, embrace_id, base_embrace, metrics_text):
    """Save filtered IBI and metrics."""
    subj_dir = os.path.join(base_embrace, embrace_id)
    out_dir = os.path.join(subj_dir, "IBI_filtrado")
    os.makedirs(out_dir, exist_ok=True)

    if len(df_embr_m) > 0:
        times = df_embr_m["time"].astype("int64") / 1e9
        ibis = df_embr_m["IBI"].to_numpy()
        t0 = float(times.iloc[0])
        offsets = times - t0

        out_csv = os.path.join(out_dir, "IBI_part01.csv")
        with open(out_csv, "w") as f:
            f.write(f"{t0:.6f},IBI\n")
            for off, val in zip(offsets, ibis):
                f.write(f"{off:.10f},{val:.9f}\n")

    out_txt = os.path.join(out_dir, "metrics.txt")
    with open(out_txt, "w") as f:
        f.write(metrics_text)


# ============================================================
# SINGLE SUBJECT PROCESSING
# ============================================================

def procesar_y_guardar(empatica_id, embrace_id,
                       base_empatica="~/TFM/EMPATICA_all",
                       base_embrace="~/TFM/2.step2_e4csv",
                       frac_tol=0.20, ratio_tol=0.15):

    base_empatica = os.path.expanduser(base_empatica)
    base_embrace = os.path.expanduser(base_embrace)

    emp_path = os.path.join(base_empatica, empatica_id, "IBI.csv")
    embr_path = os.path.join(base_embrace, embrace_id, "IBI_limpio")

    df_emp = load_ibi_empatica(emp_path)
    df_embr = load_ibi_embrace(embr_path)

    df_embr["IBI"] = hampel(df_embr["IBI"])
    df_embr["IBI"] = moving_median(df_embr["IBI"])

    best_lag, lag_corr = estimate_lag(df_emp, df_embr)
    df_emp_m, df_embr_m = match_and_filter(df_emp, df_embr,
                                           frac_tol, ratio_tol, best_lag)

    if len(df_emp_m) > 5:
        pearson, _ = pearsonr(df_emp_m["IBI"], df_embr_m["IBI"])
        rmse = np.sqrt(mean_squared_error(df_emp_m["IBI"], df_embr_m["IBI"]))
        metrics_text = (
            f"Embrace ID: {embrace_id}\n"
            f"Empatica ID: {empatica_id}\n"
            f"Lag applied: {best_lag:.3f} s\n"
            f"Raw lag correlation: {lag_corr:.3f}\n"
            f"Final Pearson correlation: {pearson:.3f}\n"
            f"RMSE: {rmse:.3f} s\n"
            f"Valid pairs: {len(df_emp_m)}\n"
        )
    else:
        pearson, rmse = np.nan, np.nan
        metrics_text = "Insufficient valid IBI pairs for metric computation."

    guardar_resultados(df_embr_m, embrace_id, base_embrace, metrics_text)

    return best_lag, lag_corr, pearson, rmse, len(df_emp_m)


# ============================================================
# BATCH PROCESSING
# ============================================================

def procesar_todos():
    base_embrace = os.path.expanduser("~/TFM/2.step2_e4csv")

    emp_emp_map = {
        "CLIDEM01": "1823370", "CLIDEM010": "1842113", "CLIDEM011": "1891004",
        "CLIDEM02": "1832743", "CLIDEM030": "1916330", "CLIDEM031": "1920720",
        "CLIDEM04": "1833368", "CLIDEM05": "1837341", "CLIDEM06": "1840660",
        "CLIDEM07": "1840605", "CLIDEM08": "1841430", "CLIDEM09": "1842162",
        "CLIDEM11": "1891004", "CLIDEM12": "1891236", "CLIDEM13": "1895868",
        "CLIDEM14": "1891785", "CLIDEM15": "1894131", "CLIDEM16": "1895849",
        "CLIDEM17": "1896649", "CLIDEM18": "1898066", "CLIDEM19": "1900189",
        "CLIDEM20": "1906938", "CLIDEM21": "1907236", "CLIDEM22": "1908525",
        "CLIDEM23": "1909384", "CLIDEM24": "1910353", "CLIDEM25": "1911867",
        "CLIDEM26": "1912600", "CLIDEM27": "1913365", "CLIDEM29": "1914654"
    }

    summary = []

    for embrace_id, empatica_id in tqdm(emp_emp_map.items()):
        try:
            res = procesar_y_guardar(empatica_id, embrace_id)
            summary.append({
                "Embrace": embrace_id,
                "Empatica": empatica_id,
                "Lag_s": res[0],
                "Lag_corr": res[1],
                "Pearson": res[2],
                "RMSE": res[3],
                "Valid_pairs": res[4]
            })
        except Exception as e:
            print(f"Error processing {embrace_id}: {e}")

    out_path = os.path.join(base_embrace, "summary_all.txt")
    pd.DataFrame(summary).to_csv(out_path, sep="\t", index=False)


# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    procesar_todos()


🚀 Procesando sujetos...



Progreso general:   0%|                                            | 0/30 [00:00<?, ?it/s]

⏱️ Lag estimado: +49.00s (corr=0.401)


Progreso general:   3%|█▏                                  | 1/30 [00:03<01:50,  3.81s/it]

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM01/IBI_filtrado/IBI_part01.csv
🧾 Métricas → /home/yasmina/TFM/2.step2_e4csv/CLIDEM01/IBI_filtrado/metrics.txt
⏱️ Lag estimado: -22.50s (corr=-0.030)


...

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM26/IBI_filtrado/IBI_part01.csv
🧾 Métricas → /home/yasmina/TFM/2.step2_e4csv/CLIDEM26/IBI_filtrado/metrics.txt
⏱️ Lag estimado: +37.00s (corr=0.532)


Progreso general:  97%|█████████████████████████████████▊ | 29/30 [02:38<00:05,  5.69s/it]

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM27/IBI_filtrado/IBI_part01.csv
🧾 Métricas → /home/yasmina/TFM/2.step2_e4csv/CLIDEM27/IBI_filtrado/metrics.txt
⏱️ Lag estimado: +43.00s (corr=0.178)


Progreso general: 100%|███████████████████████████████████| 30/30 [02:44<00:00,  5.49s/it]

✅ Guardado /home/yasmina/TFM/2.step2_e4csv/CLIDEM29/IBI_filtrado/IBI_part01.csv
🧾 Métricas → /home/yasmina/TFM/2.step2_e4csv/CLIDEM29/IBI_filtrado/metrics.txt

📊 Resumen global guardado en /home/yasmina/TFM/2.step2_e4csv/summary_all.txt
